### Change Notebook working directory

In [1]:
import os
print(os.getcwd())

/home/dai/evard/car-llm/scripts


In [2]:
%cd ..


/home/dai/evard/car-llm


/home/dai/evard/car-llm/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
print(os.getcwd())

/home/dai/evard/car-llm


### Change notebook settings

In [4]:
from IPython.display import display, HTML

display(HTML("""
<style>
.output_scroll {
    overflow-y: auto;
    max-height: 100px;
}
</style>
"""))

## 1. Fine tune dataset

In [ ]:
%%bash
llamafactory-cli train configs/finetuning_configs/qwen2vl_lora_sft_262144_res.yaml

[2025-02-05 19:22:39,720] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[INFO|2025-02-05 19:22:42] llamafactory.hparams.parser:380 >> Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.bfloat16


[INFO|configuration_utils.py:679] 2025-02-05 19:22:43,382 >> loading configuration file config.json from cache at /home/dai/evard/.cache/huggingface/hub/models--Qwen--Qwen2-VL-7B-Instruct/snapshots/a28a094eb66a9f2ac70eef346f040d8a79977472/config.json
[INFO|configuration_utils.py:746] 2025-02-05 19:22:43,387 >> Model config Qwen2VLConfig {
  "_name_or_path": "Qwen/Qwen2-VL-7B-Instruct",
  "architectures": [
    "Qwen2VLForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "image_token_id": 151655,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2_vl",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": {
    "mrope_section": [
      16,
      24,
      24
    ],
    "rope_type": "default",
    "type": "default"
  },

[INFO|2025-02-05 19:22:45] llamafactory.data.template:157 >> Add <|im_end|> to stop words.
[INFO|2025-02-05 19:22:45] llamafactory.data.loader:157 >> Loading dataset lingoQA_action.json...


Setting num_proc from 16 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 265323 examples [00:03, 68530.57 examples/s]
Converting format of dataset (num_proc=16): 100%|██████████| 265323/265323 [00:02<00:00, 112300.80 examples/s]


[INFO|2025-02-05 19:22:52] llamafactory.data.loader:157 >> Loading dataset lingoQA_scenery.json...


Setting num_proc from 16 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 148506 examples [00:01, 76753.14 examples/s]
Converting format of dataset (num_proc=16): 100%|██████████| 148506/148506 [00:01<00:00, 114608.33 examples/s]


## 3. Dataset Predictions

#### Option 1: API

Note: You must activate the http image server and the api before making requests to the api:

```bash
tmux attach -t hhtp_image_server
python -m http.server 8001
``` 
and...

```bash
tmux attach -t api
API_HOST="0.0.0.0" API_PORT="8080" llamafactory-cli api inference_configs/fixed_inference_image_resolution_589824/qwen2vl_lora_sft_lingoQA_scenery_1000.yaml
``` 

In [ ]:
from scripts.utils.inference_api import InferenceAPI

inference_pipeline = InferenceAPI(
    model_config_path="inference_configs/fixed_inference_image_resolution_589824/qwen2vl_lora_sft_lingoQA_scenery_1000.yaml",
    dataset_path="data/lingoQA_evaluation.json",
    json_output_path="data/predictions/chat_backend/fixed_inference_image_resolution_589824/qwen2vl_lora_sft_lingoQA_evaluation_500.json",
)


inference_pipeline.run(max_dataset_len=2, num_threads = 4)


#### Option 2: Chat

In [ ]:
from scripts.utils.inference_chat import InferenceChat

inference_pipeline = InferenceChat(
    model_config_path="./inference_configs/fixed_inference_image_resolution_589824/qwen2vl_lora_sft_lingoQA_action_1000.yaml",
    json_output_path="./data/predictions/chat_backend/fixed_inference_image_resolution_589824/lingoQA_action_1000_predictions.json",
    dataset_path="./data/lingoQA_action.json"
)

inference_pipeline.run(max_dataset_len=2)

#### Option 3: vLLM Backend

In [ ]:
from scripts.utils.inference_vllm import InferenceVLLM

inference_pipeline = InferenceVLLM(
    model_name_or_path="Qwen/Qwen2-VL-7B-Instruct",
    dataset="lingoQA_action",
    template="qwen2_vl",
    json_output_path="./data/predictions/chat_backend/fixed_inference_image_resolution_589824/lingoQA_action_1000_predictions.json",
    # adapter_name_or_path="./saves/qwen2_vl-7b/lora/sft/lingoQA_scenery_100" # LoRA Adapter not yet supported, but merging beforehand works
)

inference_pipeline.run(max_dataset_len=2)

### 3.1 Save Predictions

In [ ]:
inference_pipeline.save_json()

## 4. Optional: Run Benchmark

In [ ]:
%%bash


Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 35566.05 examples/s]


The overall benchmark score is 62.4%


In [ ]:
%%bash

Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 34699.23 examples/s]


The overall benchmark score is 61.8%


In [ ]:
%%bash

Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 37625.85 examples/s]


The overall benchmark score is 54.2%


In [ ]:
%%bash

Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 36249.04 examples/s]


The overall benchmark score is 53.900000000000006%


In [ ]:
%%bash



Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 38820.33 examples/s]


The overall benchmark score is 53.2%
Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 37392.39 examples/s]


The overall benchmark score is 50.6%
Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


...

The overall benchmark score is 61.9%
Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 38041.50 examples/s]


The overall benchmark score is 61.6%
Loaded 500 references.
Loaded 1000 predictions.
Matched 1000 predictions with references.
WARNING! You are evaluating on a subset of the LingoQA benchmark. Please check your input file for missing or mis-matched examples.


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 36768.92 examples/s]


The overall benchmark score is 45.9%
